In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

### opencv를 활용한 객체 검출
- opencv 내에는 yolov3의 객체를 라이브러리로 제공
- YOLO : You Only Look Once
    - 한 개의 네트워크에서 탐지를 원하는 물체의 영역(bounding box)과 이름을 표시함


1. YOLO 기본 구조
- YOLO3-416 : 416x416 이미지를 input으로 하는 모델을 의미
- 총 3개의 output layer를 갖고 있음 (82, 94, 106 레이어)
- 피처 정보들을 조합하여 찾고자 하는 객체 위치를 찾는다.
- 입력된 이미지를 일정 분할로 그리드한 다음 (그리드 기준 분할)
    - 잘게 쪼개진 이미지를 신경망을 통과시켜서 클래스 예측
    - 객체 검출만 진행하는 알고리즘이고 회귀문제로 정의하고 있다고 발표함
        - 단점 : 작은 물체에 대한 검출성능이 떨어짐 (V3)

### 처리 순서
1. 이미지 주입 
- 일반 이미지, 웹캠 스트림, 컴퓨터 캡처, 영상 등 ndarray로 치환할 수 있다면 사용 가능
2. S x S 로 이미지 나누기 (그리드)
3. 각 그리드에서 예측을 한 후 이를 종합해서 bounding box를 구성

### YOLO 사용법
- 딥러닝 프레임워크가 제공하면 해당 프레임워크를 사용


1. Darknet : YOLO 개발자가 만든 프레임워크 - YOLO를 위해 특별히 제작
- 코랩은 Darknet을 사용하면 됨
- 장점 : 빠르다. GPU 또는 CPU와 함께 사용가능
- 단점 : 리눅스에서만 호환됨


2. Darkflow : Darknet을 텐서플로우에 적용한것
- 장점 : 빠르고 GPU 또는 CPU와 함께 사용 가능하고 리눅스, 윈도우, 맥에서 호환
- 단점 : 설치 복잡


3. OpenCV : 최소 3.4.2버전 필요
- 장점 : openCV외에 설치만 필요
- 단점 : CPU에서만 작동하기 때문에 비디오를 실시간으로 처리하는 데 속도가 빠르진 않다

### OpenCV DNN YOLO 사용하기 (사용법 위주)
- yolo 사이트에서 기 학습된 모델의 weight와 conf 파일을 받아와야 함
- https://pjreddie.com/darknet/yolo/
- weight file : 훈련된 모델
- cfg file : 구성 파일 (모델 구성 파일)
- name files : 모델이 감지할 수 있는 객체명

### 사용 데이터셋
- 자동차 인식 데이터셋 (캐글 데이터)

### Open CV : 비전 처리 패키지를 활용한 YOLO 모델 구현
- 기 학습된 모델 활용
- 가중치 파일과 구성파일 활용
- cv.dnn.readNet(가중치 파일, 구성 파일)

In [2]:
!pwd

/home/lab06/Deep_Learning


In [3]:
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')  # 기 학습된 모델

### yolov3 라벨
- ms coco 데이터셋을 사용하는 구조 : classes 수가 80
- 해당 데이터셋 라벨 파일 읽어서 저장

In [4]:
# label 작성
classes = []
with open('./cardataset/coco.names.txt', 'r') as f:
    classes = [line.strip for line in f.readlines()]
classes

[<function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function str.strip>,
 <function 

In [5]:
# yolov3 층 확인
layer_names = net.getLayerNames()
# print(layer_names)

output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
output_layers

['yolo_82', 'yolo_94', 'yolo_106']

### 전체 모델 layer에서 82, 94, 106 3개의 yolo 출력을 반환함

In [6]:
# 객체 검출 이미지
img = cv2.imread('cardataset/training_images/vid_4_10000.jpg')

# 이미지 BGR 형태로 읽어오므로 RGB로 변환
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
height, width, channels = img.shape

- 네트워크(모델)에서 이미지를 바로 사용할 수 없기 때문에 모델에 맞는 이미지 파일로 변경해야 함
    - yolo는 Blob 파일을 사용
    - cv2.dnn.blobFromImage(img, scalefactor=1/255.0, size=(416, 416), swapRB=True, crop=False)

In [7]:
blob = out = cv2.dnn.blobFromImage(img, scalefactor=1/255.0, size=(416, 416), swapRB=True, crop=False)

In [8]:
print(type(out))
print(out.shape)
print(out.size)

<class 'numpy.ndarray'>
(1, 3, 416, 416)
519168


In [9]:
# 해당 이미지 모델에 주입
net.setInput(blob)

In [10]:
# 객체 검출 : forward()
outs = net.forward(output_layers)

In [11]:
outs

[array([[0.0466555 , 0.03448524, 0.28790614, ..., 0.        , 0.        ,
         0.        ],
        [0.05441702, 0.03707229, 0.22892022, ..., 0.        , 0.        ,
         0.        ],
        [0.05506101, 0.02748125, 0.96016824, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.9540115 , 0.9486618 , 0.32916814, ..., 0.        , 0.        ,
         0.        ],
        [0.96471244, 0.9620062 , 0.2737023 , ..., 0.        , 0.        ,
         0.        ],
        [0.97067446, 0.9627972 , 0.7975585 , ..., 0.        , 0.        ,
         0.        ]], dtype=float32),
 array([[0.02570593, 0.02167369, 0.05852114, ..., 0.        , 0.        ,
         0.        ],
        [0.01919862, 0.02305201, 0.25821224, ..., 0.        , 0.        ,
         0.        ],
        [0.02805037, 0.01867389, 0.08982707, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.9745448 , 0.97639394, 0.05165857, ..., 0.        , 0.        ,
         0.        

In [12]:
print('cv_outs type : list, cv_outs length : ', len(outs))

cv_outs type : list, cv_outs length :  3


In [13]:
outs[0]  # 그리드 13 X 13
outs[1]  # 그리드 26 X 26
len(outs[2][0])  # 52 X 52뒤쪽 출력일수록 이미지를 더 작게 분할해서 학습

# 각 classes별 확률(80), 바운딩박스좌표, confidence

85

### bounding box 그리기

In [14]:
# 정보를 화면에 표시
class_ids = []
confidences = []
boxes = []
n=0
for out in outs:
    n += 1
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)        
        confidence = scores[class_id]
        
        if confidence > 0.5:
            # Object detected
            print(n)
            print(class_id)                      
            print(confidence)
            print(detection[:5])
            # img를 blob 화 하면서 scaling이 진행되었으므로 이미지 원 크기에 비례해서 조정해야 함
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표(중심좌표에서 w,h의 50%만큼 감소)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

2
2
0.9849074
[0.10192333 0.56282794 0.13678345 0.08634763 0.9976587 ]
